In [2]:
from sentence_transformers import SentenceTransformer
import numpy as np, faiss, json

# загрузка
model = SentenceTransformer("intfloat/multilingual-e5-base")
index = faiss.read_index("smartwop_docs.faiss")
meta = json.load(open("smartwop_docs_meta.json", "r", encoding="utf-8"))

def search(query, k=5):
    q = model.encode([query], normalize_embeddings=True)
    q = np.asarray(q, dtype="float32")
    D, I = index.search(q, k)
    results = [{"score": float(D[0][i]), "doc": meta[int(I[0][i])]} for i in range(min(k, len(I[0])))]
    return results

print(search("Экспорт карт раскроя из SmartWOP", k=3))


[{'score': 0.8136512041091919, 'doc': {'id': 'chunk_096', 'text': 'passage: title: Export Tuning : Stage 1 title: Export Tuning : Stage 1 title: Export Tuning : Stage 1 content: SmartWOP shows you how and in which order the processing steps are carried out.'}}, {'score': 0.8057069778442383, 'doc': {'id': 'chunk_007', 'text': 'passage: title: Construction with SmartWOP вЂ“ The first cabinet title: Construction with SmartWOP вЂ“ The first cabinet title: Construction with SmartWOP вЂ“ The first cabinet content: Get a Base Volume вЂ“ SmartWOP construction is based on volumes, so for you to start working on your own design you need a Base Base volumes can be found in the Items tab (Hotkey: E). For this guide we will use a regular base. Drag the volume into your work area on screen. When you have dragged the base into the work area a window with its properties will open. Here you can name the Base, which is important, because every program and list item will be named accordingly. Here you ha

In [14]:
import os
from typing import List, Dict, Optional
from openai import OpenAI
from openai import BadRequestError

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

SYSTEM_INSTRUCTIONS = (
    "You are a helpful assistant for a CAD app (SmartWOP). "
    "Answer ONLY using the provided context. If the answer is not in context, say you don't know. "
    "Prefer step-by-step, concise instructions. Keep tool/command names verbatim."
)

def build_context_blocks(hits: List[Dict]) -> str:
    blocks = []
    for i, h in enumerate(hits, 1):
        blocks.append(f"[{i}] {h['doc']['text']}")
    return "\n\n".join(blocks)

def _create_response_safe(*, model: str, instructions: str, input_payload, 
                          max_output_tokens: int = 400, temperature: Optional[float] = None):
    kwargs = dict(
        model=model,
        instructions=instructions,
        input=input_payload,
        max_output_tokens=max_output_tokens,
        store=False,
    )
    # Пробуем с temperature, если задана
    if temperature is not None:
        kwargs["temperature"] = temperature
        try:
            return client.responses.create(**kwargs)
        except BadRequestError as e:
            # Если модель не поддерживает temperature — повторим без него
            if "Unsupported parameter" in str(e) and "temperature" in str(e):
                kwargs.pop("temperature", None)
                return client.responses.create(**kwargs)
            raise
    else:
        # Сразу без temperature
        return client.responses.create(**kwargs)

def ask_llm_with_rag(user_query: str, top_k: int = 5, 
                     model: str = "gpt-4o-mini", temperature: Optional[float] = 0.2) -> str:
    hits = search(user_query)
    context = build_context_blocks(hits)

    input_payload = [
        {
            "role": "user",
            "content": [
                {"type": "input_text",
                 "text":
                    f"User question (keep language of the user): {user_query}\n\n"
                    f"=== CONTEXT START ===\n{context}\n=== CONTEXT END ===\n\n"
                    f"Rules:\n"
                    f"- Answer in the same language as the user question.\n"
                    f"- Cite sources as [1], [2] referring to the blocks above (no external links).\n"
                    f"- If not enough info in context, try to help."
                 }
            ],
        }
    ]

    resp = _create_response_safe(
        model=model,
        instructions=SYSTEM_INSTRUCTIONS,
        input_payload=input_payload,
        max_output_tokens=300,
        temperature=temperature,   # будет удалён автоматически, если не поддерживается
    )
    return resp.output_text

# Пример вызова:
if __name__ == "__main__":
    # Пример
    q = "How to use back panels?"
    print(ask_llm_with_rag(q))


KeyboardInterrupt: 

In [19]:
# Jupyter cell: SmartWOP RAG UI (ipywidgets)
import traceback
from typing import List, Dict
from IPython.display import display, HTML, clear_output
import ipywidgets as w

# ====== UI widgets ======
title = w.HTML("<h2 style='margin:0'>SmartWOP Chat · RAG MVP</h2><p style='margin:4px 0 12px;color:#666'>Задай вопрос — получи ответ с цитированием источников [1], [2]…</p>")

q_input = w.Textarea(
    value="Wie benutze ich woodWOP Komponente?",
    placeholder="Введите вопрос (DE/EN)...",
    description="Вопрос:",
    layout=w.Layout(width="100%", height="80px")
)

model_dd = w.Dropdown(
    options=[
        ("gpt-4o-mini", "gpt-4o-mini"),
        ("gpt-5-nano", "gpt-5-nano"),
    ],
    value="gpt-4o-mini",
    description="Модель:",
    layout=w.Layout(width="300px")
)

topk_slider = w.IntSlider(
    value=5, min=1, max=10, step=1, description="Top-K:",
    continuous_update=False, readout=True, layout=w.Layout(width="300px")
)

temp_slider = w.FloatSlider(
    value=0.2, min=0.0, max=1.0, step=0.05, description="Temp:",
    continuous_update=False, readout=True, layout=w.Layout(width="300px")
)

full_toggle = w.Checkbox(value=False, description="Показывать полный текст статьи при раскрытии", indent=False)
stream_toggle = w.Checkbox(value=False, description="(опционально) Стриминг ответа в консоль", indent=False)

ask_btn = w.Button(
    description="Спросить",
    button_style="primary",
    icon="comment",
    layout=w.Layout(width="160px", height="38px")
)

status_out = w.Output(layout=w.Layout(border="1px solid #eee", padding="6px", max_height="80px", overflow_y="auto"))
ctx_out = w.Output(layout=w.Layout(border="1px solid #eee", padding="8px", max_height="250px", overflow_y="auto"))
ans_out = w.Output(layout=w.Layout(border="1px solid #eee", padding="12px"))

controls = w.HBox([model_dd, topk_slider, temp_slider])
toggles = w.HBox([full_toggle, stream_toggle])
go = w.HBox([ask_btn])

# ====== helpers ======
def _render_context(hits: List[Dict], full: bool = False) -> str:
    """Build HTML accordion with context hits."""
    if not hits:
        return "<div style='color:#999'>Контекст не найден.</div>"

    items_html = []
    for i, h in enumerate(hits, 1):
        doc = h.get('doc', {})
        title = doc.get('title', f"Doc {i}")
        text = doc.get('text', '')
        # Без f-строки, экранируем переносы заранее
        text_html = text.replace('\n', '<br>')
        items_html.append("""
        <details style='margin:8px 0;border:1px solid #eee;border-radius:8px;padding:6px;'>
          <summary style='cursor:pointer;font-weight:600'>
            [{}] {}<span style='color:#999;font-weight:400'> · score={:.2f}</span>
          </summary>
          <div style='margin-top:6px;line-height:1.45'>{}</div>
        </details>
        """.format(i, title, h.get('score', 0), text_html))
    return "".join(items_html)

def _render_answer(text: str) -> str:
    # лёгкая типографика с поддержкой жирного текста
    import re
    # Заменяем **текст** на <strong>текст</strong>
    safe = re.sub(r'\*\*(.*?)\*\*', r'<strong>\1</strong>', text)
    # Заменяем переносы строк на <br>
    safe = safe.replace("\n", "<br>")
    return "<div style='font-size:15px;line-height:1.55'>{}</div>".format(safe)

# ====== main handler ======
def on_ask_clicked(_):
    with status_out:
        clear_output()
        print("⏳ Запрашиваю контекст и ответ...")

    with ctx_out:
        clear_output()
    with ans_out:
        clear_output()

    user_q = q_input.value.strip()
    if not user_q:
        with status_out:
            clear_output()
            print("⚠️ Введите вопрос.")
        return

    try:
        # 1) Получаем контекст заранее, чтобы отрисовать
        #    Пытаемся вызвать ваш search(query, k=..., full=...) если он поддерживает full
        hits = []
        hits = search(user_q, k=topk_slider.value)

        hits = hits[:3]
        
        with ctx_out:
            clear_output()
            display(HTML("<b>Контекст:</b>"))
            display(HTML(_render_context(hits, full=full_toggle.value)))

        # 2) Ответ модели (ваша функция ask_llm_with_rag)
        #    Важно: пробрасываем top_k/model/temperature — чтобы UI реально влиял
        answer = ask_llm_with_rag(
            user_query=user_q,
            top_k=topk_slider.value,
            model=model_dd.value,
            temperature=temp_slider.value
        )

        with ans_out:
            clear_output()
            display(HTML("<b>Ответ:</b>"))
            display(HTML(_render_answer(answer)))

        with status_out:
            clear_output()
            print("✅ Готово.")

    except Exception as e:
        with status_out:
            clear_output()
            print("❌ Ошибка:")
            traceback.print_exc()

# bind
ask_btn.on_click(on_ask_clicked)

# ====== layout ======
display(
    w.VBox([
        title,
        q_input,
        controls,
        toggles,
        go,
        w.HTML("<hr style='border:none;border-top:1px solid #eee;margin:12px 0'>"),
        w.HTML("<b>Статус:</b>"),
        status_out,
        w.HTML("<b>Источники:</b>"),
        ctx_out,
        w.HTML("<b>Ответ:</b>"),
        ans_out
    ], layout=w.Layout(width="100%"))
)